<a href="https://colab.research.google.com/github/amirazaiz/fine_tuning_mobilenet/blob/main/5_fine_tuning_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Downloading VGG16

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16()

553467096/553467096 [==============================] - 24s 0us/step


In [ ]:
if (isinstance(vgg16_model, tf.keras.Sequential)):
  print("sequential")
else:
  print("not sequential")

not sequential


In [ ]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

# Fine tuning vgg16 to classify mask/nomask faces

# Removing output layer to add our customized one

In [ ]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
  model.add(layer)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

# Freezing these layers so we don't retrain them again

In [ ]:
for layer in model.layers:
  layer.trainable = False

# Adding our customized output layer

In [ ]:
model.add(Dense(units = 2, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
# params = count_params(model)
# param

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=1e-5)
model.compile(optimizer='SGD',loss='categorical_crossentropy',metrics=['accuracy'])

# Training VGG16 on our new dataset

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

Preprocessing data

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                  )
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/dataset/face_mask_detection',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')

Found 194 images belonging to 2 classes.


In [ ]:
# train_datagen = ImageDataGenerator(
#                                    zoom_range=0.15,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    shear_range=0.15)
# training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/dataset/face_mask_detection',
#                                                     target_size=(224, 224),
#                                                     batch_size=32,
#                                                     shuffle=True,
#                                                     class_mode='categorical')

Found 194 images belonging to 2 classes.


In [ ]:
training_set.class_indices

{'with_mask': 0, 'without_mask': 1}

In [ ]:
type(training_set)

keras.preprocessing.image.DirectoryIterator

In [ ]:
model.fit(training_set, epochs = 25)

Epoch 1/25
20/20 [==============================] - 6s 223ms/step - loss: 7.3183 - accuracy: 0.4691
Epoch 2/25
20/20 [==============================] - 3s 148ms/step - loss: 4.3168 - accuracy: 0.5722
Epoch 3/25
20/20 [==============================] - 3s 147ms/step - loss: 4.0692 - accuracy: 0.5670
Epoch 4/25
20/20 [==============================] - 3s 152ms/step - loss: 3.9965 - accuracy: 0.5206
Epoch 5/25
20/20 [==============================] - 3s 162ms/step - loss: 3.2721 - accuracy: 0.5876
Epoch 6/25
20/20 [==============================] - 3s 140ms/step - loss: 3.1349 - accuracy: 0.6237
Epoch 7/25
20/20 [==============================] - 3s 149ms/step - loss: 1.0307 - accuracy: 0.8196
Epoch 8/25
20/20 [==============================] - 3s 166ms/step - loss: 1.3546 - accuracy: 0.7371
Epoch 9/25
20/20 [==============================] - 3s 140ms/step - loss: 1.8788 - accuracy: 0.7113
Epoch 10/25
20/20 [==============================] - 3s 145ms/step - loss: 0.3878 - accuracy: 0.9072

# Evaluating Model

In [ ]:
# model.evaluate_generator(test_data)

In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(model.history[“acc”])
# plt.plot(model.history[‘val_acc’])
# plt.plot(model.history[‘loss’])
# plt.plot(hist.history[‘val_loss’])
# plt.title(“model accuracy”)
# plt.ylabel(“Accuracy”)
# plt.xlabel(“Epoch”)
# plt.legend([“Accuracy”,”Validation Accuracy”,”loss”,”Validation Loss”])
# plt.show()

# Saving model

In [ ]:
model.save('/content/drive/My Drive/weights.h5')


downloading model to local machine

In [ ]:
from google.colab import files
files.download('/content/drive/My Drive/weights.h5')


# Using saved model

In [ ]:
model.load_weights('/content/drive/MyDrive/weights.h5')

In [ ]:
import cv2
import numpy as np
image = cv2.imread('/content/drive/MyDrive/dataset/benzema.png')

In [ ]:
image = cv2.resize(image, (224,224))

# Remember class_indices: {"with mask" ,"without mask"}

In [ ]:
preds = model.predict(np.expand_dims(image, axis=0))[0]

print('predicted Label',preds)

1/1 [==============================] - 0s 23ms/step
predicted Label [4.0400786e-28 1.0000000e+00]


In [ ]:
image_with_mask = cv2.imread('/content/drive/MyDrive/dataset/test.jpg')

The np.expand_dims(image_with_mask, axis=0) function call adds an extra dimension to the input image image_with_mask, effectively converting it from a 2D image to a 3D array with a batch size of 1. This is necessary because Keras models expect input data to be in the form of batches, even if the batch size is just 1.

The resulting 3D array is then passed to the model.predict() function, which returns a prediction for the input data. The [0] at the end of the code snippet is used to extract the prediction for the first (and only) image in the batch.

Overall, the code snippet is used to make a prediction on a single image with a mask using a Keras model.

In [ ]:
image_with_mask = cv2.resize(image_with_mask, (224,224))

In [ ]:
preds = model.predict(np.expand_dims(image_with_mask, axis=0))[0]

print('predicted Label',preds)

1/1 [==============================] - 0s 116ms/step
predicted Label [1.0000000e+00 1.9022287e-19]
